In [18]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import sqlite3
from datetime import date
from datetime import datetime as dt
import hashlib
import sys
import os
import locale
import re
import requests as rq
import wbdata as wb
from contextlib import contextmanager
import pytz
from datetime import timedelta
from currency_converter import CurrencyConverter
cc=CurrencyConverter()

project_dir = '/home/mi/Dropbox/prog/python/tradeDB'
os.chdir(project_dir)

from importlib import reload
from workers import sql
from workers import api
from workers.common import read_json, hash_table
import tradeDB

In [2]:
SECTORS = {
            "INDEXES": {"file": "./assets/indexes.jsonc"},
            "STOCK": {"file": "./assets/stock.jsonc"},
            "ETF": {"file": "./assets/etf.jsonc"},
            "COMODITIES": {"file": "./assets/comodities.jsonc"},
        }
for k in SECTORS:
    SECTORS[k]["data"] = read_json( # type: ignore
        SECTORS[k]["file"]
    )

In [3]:
trade= tradeDB.Trader()
db_file = trade.db

In [4]:
reload(sql)

<module 'workers.sql' from '/home/mi/Dropbox/prog/python/tradeDB/workers/sql.py'>

In [21]:
cur=sql.get(db_file=db_file,tab="GEO",get=['currency_code'],search=['%'],cols=['currency_code'])
cur=cur['currency_code']['currency_code'].to_list()


In [23]:
cc.convert(100,'USD')

94.8856627763545

In [26]:
cc._rates.keys()


dict_keys(['USD', 'JPY', 'BGN', 'CZK', 'DKK', 'GBP', 'HUF', 'PLN', 'RON', 'SEK', 'CHF', 'ISK', 'NOK', 'TRY', 'AUD', 'BRL', 'CAD', 'CNY', 'HKD', 'IDR', 'ILS', 'INR', 'KRW', 'MXN', 'MYR', 'NZD', 'PHP', 'SGD', 'THB', 'ZAR', 'HRK', 'RUB', 'LTL', 'LVL', 'EEK', 'SKK', 'CYP', 'MTL', 'SIT', 'ROL', 'TRL'])

In [30]:
import pandasdmx as sdmx
ecb=sdmx.Request('ECB')

In [40]:
cur=ecb.dataflow('EXR').dataflow.EXR.structure.dimensions.components

In [48]:
exr=sdmx.to_pandas(cur[1].local_representation.enumerated).index.to_list()

In [ ]:
[]